# Creating test spreadsheet
## Import pandas and load files

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir='/Users/davecash/Data/IDEAS/sample'
xls_demo=os.path.join(data_dir,'GENFI_DEMOGRAPHICS_DF3_FINAL_BLINDED.xlsx')
df_demo=pd.read_excel(xls_demo)
xls_img=os.path.join(data_dir,'GENFI_IMAGING_DF3_FINAL_BLINDED.xlsx')
df_img=pd.read_excel(xls_img)

## Combining data
Join the data, get rid of unneeded variables and keep first visit that has both demographics and imaging

In [3]:
df_combined=pd.merge(df_img,df_demo,on=['Blinded Code','Visit'])

Differentiate genetic status from symptoms

In [4]:
df_combined['Affected'] = np.where(df_combined['Genetic status 1']=='A', 1, 0)
df_combined['Mutation'] = df_combined['Genetic status 2'] >0

Categorize mutation type and get rid of any rare mutations

In [5]:
df_combined['MutationType']=pd.Categorical(df_combined['Genetic Group'],categories=['C9orf72','GRN','MAPT'])
df_combined = df_combined.dropna(subset=['MutationType','Age at visit','DRC_QC','Scanner'])

Now it's time to get rid of some missing values. Start by assuming TIV constant over time and just keeping first value

In [6]:
df_combined['TIV'] = df_combined.groupby(['Blinded Code'])["TIV mm3"].fillna(method="ffill")

Remove a bunch of columns that we don't need.

In [7]:
df_combined = df_combined.drop(columns=['Date of scan','Blinded Site_x','TIV mm3','1  Orbitofrontal LEFT', 
                                        '2.  DLPFC LEFT', '3.  VMPFC LEFT', '4.  Motor LEFT', '5.  Opercular LEFT', 
                                        '6.  FRP LEFT', '7.  Medial Temp LEFT', '8.  Lateral Temp  LEFT', 
                                        '9.  Temporal Pole LEFT', '10.  Supra Temp LEFT', '11.  Medial Parietal LEFT', 
                                        '12.  Lateral Parietal LEFT', '13.  Sensory LEFT', '14.  Medial occ LEFT', 
                                        '15.  Lateral Occ LEFT', '16.  Anterior Cing LEFT', '17.  Middle Cing LEFT', 
                                        '18.  Posterior Cing LEFT', '19.  Ant Insula  LEFT', '20.  Post Insula LEFT', 
                                        '1  Orbitofrontal RIGHT', '2.  DLPFC RIGHT', '3.  VMPFC RIGHT', '4.  Motor RIGHT',
                                        '5.  Opercular RIGHT', '6.  FRP RIGHT', '7.  Medial Temp RIGHT', 
                                        '8.  Lateral Temp  RIGHT', '9.  Temporal Pole RIGHT', '10.  Supra Temp RIGHT', 
                                        '11.  Medial Parietal RIGHT', '12.  Lateral Parietal RIGHT', '13.  Sensory RIGHT', 
                                        '14.  Medial occ RIGHT', '15.  Lateral Occ RIGHT', '16.  Anterior Cing RIGHT', 
                                        '17.  Middle Cing RIGHT', '18.  Posterior Cing RIGHT', '19.  Ant Insula  RIGHT', 
                                        '20.  Post Insula RIGHT','Genetic Group', 'Blinded Family', 'Genetic status 1', 
                                        'Genetic status 2', 'Date of assessment','Handedness', 'Employment', 'Ethnicity'])

Keep only images that passed QC for GIF

In [8]:
df_combined=df_combined[df_combined['QC_include in GIF']==1]
df_combined

,Blinded Code,Visit,Scanner,QC_include in VBM,QC_include in GIF,DRC_QC,Right Accumbens Area,Left Accumbens Area,Right Amygdala,Left Amygdala,...,Total Cerebellum,Blinded Site_y,Age at visit,Gender,Education,EYO,Affected,Mutation,MutationType,TIV
0,C9ORF001,1.0,Siemens Trio 3T,1.0,1.0,1,444.483,432.470,1488.280,1437.56,...,102781.0824,GENFI_AA,41.820671,0,12,-22.179329,0,False,C9orf72,1273224.2
1,C9ORF001,2.0,Siemens Trio 3T,1.0,1.0,0.5,445.818,472.514,1469.600,1461.59,...,102973.1936,GENFI_AA,42.806297,0,12,-21.193703,0,False,C9orf72,1273224.2
2,C9ORF002,1.0,Siemens Trio 3T,1.0,1.0,1,460.938,480.469,1634.770,1535.16,...,100718.7498,GENFI_AA,43.550992,0,11,-20.449008,0,False,C9orf72,1292113.7
3,C9ORF002,2.0,Siemens Trio 3T,1.0,1.0,1,429.800,439.144,1607.080,1412.20,...,104434.8356,GENFI_AA,44.591376,0,11,-19.408624,0,False,C9orf72,1292113.7
4,C9ORF002,11.0,Siemens Trio 3T,1.0,1.0,1,400.435,460.500,1588.390,1390.84,...,103123.9806,GENFI_AA,46.721424,0,11,-17.278576,0,False,C9orf72,1292113.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,MAPT111,11.0,Siemens Trio 3T,1.0,1.0,1,525.904,619.339,1964.800,2132.98,...,115812.3731,GENFI_VW,37.839836,1,13,-12.160164,0,True,MAPT,1709794.8
1183,MAPT117,11.0,Philips 3T,1.0,1.0,1,369.733,395.093,951.695,1002.42,...,83853.2448,GENFI_BM,56.331280,0,15,1.331280,1,True,MAPT,1129471.7
1184,MAPT118,11.0,Siemens Skyra 3T,1.0,1.0,1,515.227,559.274,1501.630,1624.43,...,101205.9922,GENFI_LL,33.486653,0,12,-28.513347,0,False,MAPT,1313894.6
1185,MAPT119,11.0,Siemens Skyra 3T,1.0,1.0,1,517.896,540.588,1659.140,1655.13,...,123535.6880,GENFI_LL,22.937714,0,14,-39.062286,0,False,MAPT,1371629.0


Keep only one timepoint

In [30]:
df_xsec=df_combined.drop_duplicates(subset='Blinded Code')
df_xsec

,Blinded Code,Visit,Scanner,QC_include in VBM,QC_include in GIF,DRC_QC,Right Accumbens Area,Left Accumbens Area,Right Amygdala,Left Amygdala,...,Total Cerebellum,Blinded Site_y,Age at visit,Gender,Education,EYO,Affected,Mutation,MutationType,TIV
0,C9ORF001,1.0,Siemens Trio 3T,1.0,1.0,1,444.483,432.470,1488.280,1437.56,...,102781.0824,GENFI_AA,41.820671,0,12,-22.179329,0,False,C9orf72,1273224.2
2,C9ORF002,1.0,Siemens Trio 3T,1.0,1.0,1,460.938,480.469,1634.770,1535.16,...,100718.7498,GENFI_AA,43.550992,0,11,-20.449008,0,False,C9orf72,1292113.7
5,C9ORF003,1.0,Siemens Trio 3T,0.5,1.0,0,509.000,568.000,1700.000,1649.00,...,103085.0000,GENFI_GX,45.319644,0,17,-14.280356,0,False,C9orf72,1333397.3
9,C9ORF004,1.0,Siemens Trio 3T,1.0,1.0,1,528.574,543.257,1565.700,1689.84,...,97718.1944,GENFI_MB,65.678303,0,16,5.678303,0,False,C9orf72,1313058.6
10,C9ORF005,1.0,Siemens 1.5T,0.5,1.0,0,511.875,571.875,1571.250,1627.50,...,83060.7000,GENFI_IS,69.705681,0,8,5.005681,0,False,C9orf72,1345118.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,MAPT111,11.0,Siemens Trio 3T,1.0,1.0,1,525.904,619.339,1964.800,2132.98,...,115812.3731,GENFI_VW,37.839836,1,13,-12.160164,0,True,MAPT,1709794.8
1183,MAPT117,11.0,Philips 3T,1.0,1.0,1,369.733,395.093,951.695,1002.42,...,83853.2448,GENFI_BM,56.331280,0,15,1.331280,1,True,MAPT,1129471.7
1184,MAPT118,11.0,Siemens Skyra 3T,1.0,1.0,1,515.227,559.274,1501.630,1624.43,...,101205.9922,GENFI_LL,33.486653,0,12,-28.513347,0,False,MAPT,1313894.6
1185,MAPT119,11.0,Siemens Skyra 3T,1.0,1.0,1,517.896,540.588,1659.140,1655.13,...,123535.6880,GENFI_LL,22.937714,0,14,-39.062286,0,False,MAPT,1371629.0


Find all of the scanners, sort and assign a new identifying number to use instead of the official GENFI one.

In [31]:
site_list=df_xsec['Blinded Site_y'].drop_duplicates()
site_list=site_list.sort_values(ignore_index=True)
site_list = site_list.reset_index(level=0)
site_list.set_index('Blinded Site_y',inplace=True)
site_list

,index
Blinded Site_y,
GENFI_AA,0
GENFI_AD,1
GENFI_BK,2
GENFI_BM,3
GENFI_CT,4
GENFI_CV,5
GENFI_EN,6
GENFI_GX,7
GENFI_IS,8


In [32]:
df_xsec = pd.merge(df_xsec,site_list,how='left',left_on='Blinded Site_y',right_index=True)

In [33]:
df_xsec

,Blinded Code,Visit,Scanner,QC_include in VBM,QC_include in GIF,DRC_QC,Right Accumbens Area,Left Accumbens Area,Right Amygdala,Left Amygdala,...,Blinded Site_y,Age at visit,Gender,Education,EYO,Affected,Mutation,MutationType,TIV,index
0,C9ORF001,1.0,Siemens Trio 3T,1.0,1.0,1,444.483,432.470,1488.280,1437.56,...,GENFI_AA,41.820671,0,12,-22.179329,0,False,C9orf72,1273224.2,0
2,C9ORF002,1.0,Siemens Trio 3T,1.0,1.0,1,460.938,480.469,1634.770,1535.16,...,GENFI_AA,43.550992,0,11,-20.449008,0,False,C9orf72,1292113.7,0
5,C9ORF003,1.0,Siemens Trio 3T,0.5,1.0,0,509.000,568.000,1700.000,1649.00,...,GENFI_GX,45.319644,0,17,-14.280356,0,False,C9orf72,1333397.3,7
9,C9ORF004,1.0,Siemens Trio 3T,1.0,1.0,1,528.574,543.257,1565.700,1689.84,...,GENFI_MB,65.678303,0,16,5.678303,0,False,C9orf72,1313058.6,13
10,C9ORF005,1.0,Siemens 1.5T,0.5,1.0,0,511.875,571.875,1571.250,1627.50,...,GENFI_IS,69.705681,0,8,5.005681,0,False,C9orf72,1345118.6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,MAPT111,11.0,Siemens Trio 3T,1.0,1.0,1,525.904,619.339,1964.800,2132.98,...,GENFI_VW,37.839836,1,13,-12.160164,0,True,MAPT,1709794.8,19
1183,MAPT117,11.0,Philips 3T,1.0,1.0,1,369.733,395.093,951.695,1002.42,...,GENFI_BM,56.331280,0,15,1.331280,1,True,MAPT,1129471.7,3
1184,MAPT118,11.0,Siemens Skyra 3T,1.0,1.0,1,515.227,559.274,1501.630,1624.43,...,GENFI_LL,33.486653,0,12,-28.513347,0,False,MAPT,1313894.6,12
1185,MAPT119,11.0,Siemens Skyra 3T,1.0,1.0,1,517.896,540.588,1659.140,1655.13,...,GENFI_LL,22.937714,0,14,-39.062286,0,False,MAPT,1371629.0,12


In [34]:
df_xsec.rename(columns={'index':'Site'}, inplace=True)

Finally blind the blinded code and re-sort on the randomcode

In [44]:
new_id=1+np.arange(len(df_xsec.index))
np.random.shuffle(new_id)

In [45]:
df_xsec['New Code']=new_id

In [46]:
df_xsec['ID']=df_xsec['New Code'].map('GENFI{:03d}'.format)

In [47]:
df_xsec=df_xsec[['ID','Site','Scanner', 'Age at visit', 'Gender',
                 'Education', 'EYO', 'Affected', 'Mutation', 'MutationType', 'TIV',
                 'Right Accumbens Area','Left Accumbens Area', 'Right Amygdala', 
                 'Left Amygdala', 'Pons','Brain Stem', 'Right Caudate', 'Left Caudate',
                 'Right Hippocampus','Left Hippocampus', 'Right Pallidum', 'Left Pallidum',
                 'Right Putamen','Left Putamen', 'Right Thalamus Proper', 'Left Thalamus Proper',
                 'Total_Brain', 'Frontal lobe volume', 'Temporal lobe volume',
                 'Parietal lobe volume', 'Occipital lobe volume', 'Cingulate volume',
                 'Insula volume', 'Left Frontal lobe volume', 'Right Frontal lobe volume', 
                 'Left Temporal lobe volume','Right Temporal lobe volume', 'Left Parietal lobe volume',
                 'Right Parietal lobe volume', 'Left Occipital lobe volume',
                 'Right Occipital lobe volume', 'Left Cingulate volume', 'Right Cingulate volume', 
                 'Left Insula volume', 'Right Insula volume','Total Cerebellum']]
df_xsec=df_xsec.set_index('ID',drop=True)

In [48]:
df_xsec=df_xsec.sort_index()
df_xsec

,Site,Scanner,Age at visit,Gender,Education,EYO,Affected,Mutation,MutationType,TIV,...,Right Temporal lobe volume,Left Parietal lobe volume,Right Parietal lobe volume,Left Occipital lobe volume,Right Occipital lobe volume,Left Cingulate volume,Right Cingulate volume,Left Insula volume,Right Insula volume,Total Cerebellum
ID,,,,,,,,,,,,,,,,,,,,,
GENFI001,15,Siemens Skyra 3T,68.353183,1,11,-0.446817,1,True,C9orf72,1682172.5,...,62905.652,49489.819,49504.431,36287.44,38645.990,16666.11,15392.73,5407.21,4933.37,105070.1827
GENFI002,21,Siemens Trio 3T,55.972621,0,16,6.572621,1,True,MAPT,1369226.1,...,52472.950,45265.191,44123.993,33258.81,34888.590,14100.65,12544.31,3646.63,3918.92,104926.0028
GENFI003,17,Philips 3T,49.223819,0,15,-13.576181,0,True,GRN,1307454.3,...,56293.390,47800.155,46031.515,33930.32,34201.300,14076.68,12421.53,4412.81,4806.57,96191.5656
GENFI004,3,Philips 3T,49.519507,0,14,-8.780493,0,True,GRN,1186448.5,...,48975.715,38435.003,35127.408,33333.47,31973.340,12420.13,10659.55,4064.41,4192.54,86732.6708
GENFI005,1,Philips 3T,28.271047,1,13,-36.728953,0,False,C9orf72,1532213.1,...,65532.670,49663.459,48583.571,37733.07,38209.600,17087.94,15335.36,5535.36,5455.27,102753.1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GENFI620,3,Philips 3T,51.589322,0,14,-6.710678,0,True,GRN,1609958.8,...,65728.620,55101.062,52778.639,43993.08,44484.280,18576.15,17039.82,6274.81,6744.65,118775.5949
GENFI621,5,GE 3T,71.594798,1,20,7.594798,1,True,C9orf72,1550875.2,...,53483.650,37185.179,33022.687,29632.55,29984.069,12442.92,10613.55,4301.12,4000.35,97591.4618
GENFI622,12,Siemens Skyra 3T,46.592745,1,11,-4.907255,0,False,MAPT,1586157.5,...,65698.110,50185.197,50090.460,40614.79,42754.470,17432.28,15599.62,5448.59,6067.93,113102.9497


In [56]:
df_xsec=df_xsec.round(1)
df_xsec=df_xsec.round({'TIV':0})

In [57]:
df_xsec.to_excel(os.path.join(data_dir,'GENFI_DEMON_SPREADSHEET.xlsx'))